In [ ]:
import numpy as np

In [ ]:


def huber_derivative(r, gamma):
    """Computes h_0^gamma (derivative of Huber loss)"""
    return np.where(np.abs(r) <= gamma, r, gamma * np.sign(r))

def huber_psi(r, gamma):
    """Computes psi_gamma (second derivative of Huber loss)"""
    return np.where(np.abs(r) <= gamma, 1, 0)

def sncd(X, y, lambda_, alpha, tau=0.5, gamma=1.0, max_iter=100):
    """Solves penalized Huber regression using Sequential Negative Coordinate Descent (SNCD)"""
    n, p = X.shape
    beta = np.zeros(p)
    s = np.zeros(p)
    beta_0 = 0
    
    for _ in range(max_iter):
        # Compute residuals
        r = y - (beta_0 + X @ beta)
        
        # Update beta_0
        beta_0 += (np.sum(huber_derivative(r, gamma)) + 2 * tau - 1) / np.sum(huber_psi(r, gamma))
        
        for j in range(p):
            r = y - (beta_0 + X @ beta)  # Recompute residuals
            
            num = np.sum(huber_derivative(r, gamma) * X[:, j]) + (2 * tau - 1)
            den = np.sum(huber_psi(r, gamma) * X[:, j] ** 2) / (2 * n) + lambda_ * (1 - alpha)
            
            if np.abs(beta[j] + s[j]) > 1:
                beta[j] += (num / (2 * n) - lambda_ * alpha * np.sign(beta[j] + s[j]) - lambda_ * (1 - alpha) * beta[j]) / den
                s[j] = np.sign(beta[j] + s[j])
            else:
                beta[j] = 0
                s[j] = (num / (2 * n) + beta[j] * (den / lambda_ * alpha))
    
    return beta_0, beta
